In [0]:
enriched = spark.table("milbom_silver.bakehouse_transactions_enrich")

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS milbom_gold;

In [0]:
gold_kpi_revenue_period = "milbom_gold.kpi_revenue_period"
gold_kpi_product_category = "milbom_gold.kpi_sales_product_category"
gold_kpi_ticket_customer = "milbom_gold.kpi_ticket_customer"
gold_kpi_store_perf = "milbom_gold.kpi_store_performance"

In [0]:
from pyspark.sql import DataFrame
from pyspark.sql import functions as F

def add_load_metadata(
    df: DataFrame,
    load_ts_col: str = "load_ts",
    load_date_col: str = "load_date",
) -> DataFrame:
    return (
        df.withColumn(load_ts_col, F.current_timestamp())
          .withColumn(load_date_col, F.to_date(F.col(load_ts_col)))
    )

In [0]:
def write_delta(
    df: DataFrame,
    full_table_name: str,
    mode: str = "overwrite",
    partition_cols=None,
    merge_schema=True,
) -> None:
    writer = df.write.format("delta").mode(mode)
    if merge_schema:
        writer = writer.option("mergeSchema", "true")
    if partition_cols:
        writer = writer.partitionBy(*partition_cols)
    writer.saveAsTable(full_table_name)

In [0]:
# KPI 1: Faturamento por dia e por mês
kpi_day = (
    enriched.groupBy(F.to_date("dateTime").alias("date"))
        .agg(
            F.sum("totalPrice").alias("total_revenue"),
            F.countDistinct("transactionID").alias("total_transactions")
        )
        .withColumn("period_grain", F.lit("day"))
)

kpi_month = (
    enriched.groupBy("year", "month")
        .agg(
            F.sum("totalPrice").alias("total_revenue"),
            F.countDistinct("transactionID").alias("total_transactions")
        )
        .withColumn("period_grain", F.lit("month"))
)

kpi_revenue = (
    kpi_day.selectExpr(
        "cast(null as int) as year",
        "cast(null as int) as month",
        "date",
        "period_grain",
        "total_revenue",
        "total_transactions"
    )
    .unionByName(
        kpi_month.selectExpr(
            "year",
            "month",
            "cast(null as date) as date",
            "period_grain",
            "total_revenue",
            "total_transactions"
        )
    )
)
kpi_revenue = add_load_metadata(kpi_revenue)
write_delta(kpi_revenue, gold_kpi_revenue_period, mode="append")

# KPI 2: Vendas por produto e categoria
kpi_prod_cat = (
    enriched.groupBy("product", "category")
        .agg(
            F.sum("quantity").alias("total_qty"),
            F.sum("totalPrice").alias("total_revenue"),
            F.countDistinct("transactionID").alias("total_transactions")
        )
)
kpi_prod_cat = add_load_metadata(kpi_prod_cat)
write_delta(kpi_prod_cat, gold_kpi_product_category, mode="overwrite")

# KPI 3: Ticket médio por cliente
kpi_ticket_customer = (
    enriched.groupBy("customerID", "customer_city", "customer_state", "customer_country")
        .agg(
            F.sum("totalPrice").alias("customer_revenue"),
            F.countDistinct("transactionID").alias("customer_transactions")
        )
        .withColumn(
            "avg_ticket",
            F.round(F.col("customer_revenue") / F.col("customer_transactions"), 2)
        )
)
kpi_ticket_customer = add_load_metadata(kpi_ticket_customer)
write_delta(kpi_ticket_customer, gold_kpi_ticket_customer, mode="overwrite")

# KPI 4: Performance por loja
kpi_store_perf = (
    enriched.groupBy("franchiseID", "store_name", "store_city", "store_country", "store_size")
        .agg(
            F.sum("totalPrice").alias("store_revenue"),
            F.countDistinct("transactionID").alias("store_transactions"),
            F.countDistinct("customerID").alias("unique_customers")
        )
        .withColumn(
            "avg_ticket_store",
            F.round(F.col("store_revenue") / F.col("store_transactions"), 2)
        )
)
kpi_store_perf = add_load_metadata(kpi_store_perf)
write_delta(kpi_store_perf, gold_kpi_store_perf, mode="overwrite")
